# NLP Clinical Trials - Creating a Question/Answer Machine Simular to Westworld Robheum

What really needs to be done:

1.   Load the data - done 
2.   Create the vectors file - on the way 
3.   Utilize the vectorization for creationg sentence embedding index for impliarity  
3.   Process sentence embedding and index



In [0]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3019051 sha256=088ebb9b60434f6266b9e9147150f5fbd807abce222555f30b5bab3fbfea230e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [0]:
#Loading Section 
import os
import shutil
import pandas as pd
import sqlite3

#Load Data Section

In [0]:
#Load the data through google drive
#Section for Google Drive OAuth
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

try:
    os.stat("./data/")
except:
    os.mkdir("./data/")  
shutil.copy("../content/gdrive/My Drive/articles.sqlite", "data")
 

'data/articles.sqlite'

In [0]:
#load databbase into memory
# Connect to database
db = sqlite3.connect("data/articles.sqlite")
pd.set_option("max_colwidth", 125)

#Analyze Data Section

In [0]:
articles = pd.read_sql_query("select Id,Title,Tags,Size,Method from articles where Size > 0 LIMIT 5", db)
articles.head()


,Id,Title,Tags,Size,Method
0,8e9605de5913710c88b1107fd65dcc26ab372805,Comparison of respiratory pathogen yields from Nasopharyngeal/Oropharyngeal swabs and sputum specimens collected from hos...,COVID-19,2019,We compared pathogen yields from NP/OP swabs and sputum specimens from patients ≥18 years hospitalized with ALRI in rural...
1,d958168df85240e544a918d843a14e887dc41d2b,Note from the editors: novel coronavirus (2019-nCoV),COVID-19,20,None
2,4b36607cdbc54f8006161a9a1839489dd0a51269,A qualitative study of zoonotic risk factors among rural communities in southern China,COVID-19,88,"METHODS: Residents in rural communities of Yunnan, Guangxi and Guangdong provinces were recruited and enrolled in this st..."
3,d99dbae98cc9705d9b5674bb6eb66560b4434305,The third coronavirus epidemic in the third millennium: what’s next?,COVID-19,2500,THe eMeRGence oF covId-19 cAused by sARs-cov-2
4,53a3bfcb863542f8e6422ff2ae753f204fddf83d,The impact of point-of-care testing for influenza A and B on patient flow and management in a medical assessment unit of ...,COVID-19,54,"Data was gathered from the Hospital LIS and Patient Information Management System (PIMS, iSoft) which were retrospectivel..."


#Build the embeddings 


In [0]:
#Define some tokenizer objects:
"""
Text tokenization methods
"""

import re
import string

class Tokenizer(object):
    """
    Text tokenization methods
    """

    # English Stop Word List (Standard stop words used by Apache Lucene)
    STOP_WORDS = {"a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", "it",
                  "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", "these",
                  "they", "this", "to", "was", "will", "with"}

  
    def tokenize(text):
        """
        Tokenizes input text into a list of tokens. Filters tokens that match a specific pattern and removes stop words.
        Args:
            text: input text
        Returns:
            list of tokens
        """

        # Convert to all lowercase, split on whitespace, strip punctuation
        tokens = [token.strip(string.punctuation) for token in text.lower().split()]

        # Tokenize on alphanumeric strings.
        # Require strings to be at least 2 characters long.
        # Require at least 1 alpha character in string.
        return [token for token in tokens if re.match(r"^\d*[a-z][\-.0-9:_a-z]{1,}$", token) and token not in Tokenizer.STOP_WORDS]



class RowIterator(object):
    """
    Iterates over rows in a database query. Allows for multiple iterations.
    """

    def __init__(self, dbfile):
        """
        Initializes RowIterator.
        Args:
            dbfile: path to SQLite file
        """

        # Store database file
        self.dbfile = dbfile

        self.rows = self.stream(self.dbfile)

    def __iter__(self):
        """
        Creates a database query generator.
        Returns:
            generator
        """

        # reset the generator
        self.rows = self.stream(self.dbfile)
        return self

    def __next__(self):
        """
        Gets the next result in the current generator.
        Returns:
            tokens
        """

        result = next(self.rows)
        if result is None:
            raise StopIteration
        else:
            return result

    def stream(self, dbfile):
        """
        Connects to SQLite file at dbfile and yields parsed tokens for each row.
        Args:
            dbfile:
        """

        # Connection to database file
        db = sqlite3.connect(dbfile)
        cur = db.cursor()

        cur.execute("SELECT Text FROM sections")

        count = 0
        for section in cur:
            # Tokenize text
            tokens = Tokenizer.tokenize(section[0])

            count += 1
            if count % 1000 == 0:
                print("Streamed %d documents" % (count), end="\r")

            # Skip documents with no tokens parsed
            if tokens:
                yield tokens

        print("Iterated over %d total rows" % (count))

        # Free database resources
        db.close()


In [0]:
#First let's get all the text from the db
import tempfile
# Connection to database file
cur = db.cursor()

cur.execute("SELECT Text FROM sections")

count = 0
#print(cur)
dbfile = "data/articles.sqlite"

#loop to write tokens to a temporary file 
with tempfile.NamedTemporaryFile(mode="w", suffix=".txt", delete=False) as output:
          # Save file path
          tokens = output.name

          for row in RowIterator(dbfile):
              output.write(" ".join(row) + "\n")








  #this is the text what needs to be done: 
    #1st is to tokenize all the text - write tokens to a temporary file 
    #turn tokenze into vectors - we may want to stream the tokens to a temporary file 
    #create model using the vectors 


Iterated over 11390230 total rows


In [0]:
#Training the vectors 
# Train on tokens file using largest dimension size

import fasttext
import fasttext.util
size = 300 
mincount = 3
model = fasttext.train_unsupervised(tokens, dim=size, minCount=mincount)

In [0]:
#Next is to take the vectors in the model and writing it out to a file and converting 
# Output file path

path = "data/covid-fasttext-vectors"

# Output vectors in vec/txt format
with open(path + ".txt", "w") as output:
    words = model.get_words()
    output.write("%d %d\n" % (len(words), model.get_dimension()))

    for word in words:
        # Skip end of line token
        if word != "</s>":
            vector = model.get_word_vector(word)
            data = ""
            for v in vector:
                data += " " + str(v)

            output.write(word + data + "\n")

# Build magnitude vectors database
print("Converting vectors to magnitude format")
converter.convert(path + ".txt", path + ".magnitude", subword=True)

In [0]:
for section in cur:
      # Tokenize text
      tokens = Tokenizer.tokenize(section[0])

      count += 1
      if count % 1000 == 0:
          print("Streamed %d documents" % (count), end="\r")

      # Skip documents with no tokens parsed
      if tokens:
          yield tokens
print("Iterated over %d total rows" % (count))

In [0]:

  
import zipfile
#Load subsequent required files 
#Fast Vectors Load - Precomputed Word Vectors the document 
# https://www.kaggle.com/davidmezzetti/cord19-fasttext-vectors
#shutil.copy("../content/gdrive/My Drive/cord19-fasttext-vectors.zip", "data")
with zipfile.ZipFile('./data/cord19-fasttext-vectors.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/')

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [0]:
#The sections related to all the articles and all the sentences
sections = pd.read_sql_query("select * from sections LIMIT 5", db)
#sections.head()


Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis


In [0]:
#Fast Vectors Load - Precomputed Word Vectors the document 
# https://www.kaggle.com/davidmezzetti/cord19-fasttext-vectors

In [0]:
#Compute code
!pip install git+https://github.com/neuml/cord19q

#Using the clinical study attribute models created 

In [0]:
#Not really needed but nice to see how easy it is to move over the clinical study attributes models
import pickle
load_model = pickle.load(open('abc.sav','rb'))
sections = pd.read_sql_query("select * from sections LIMIT 5", db)
#sections.head()
tempsentence = sections.iloc[0]["Text"]
#Inference Demo
#One Sentence Test of Prediction
tempsentence = pd.Series(tempsentence)
#Run Predictions 
#prediction_all= load_model.predict(testsentences)
predict_one = load_model.predict(tempsentence)
print(predict_one)

[0]


#Building the embedding Index


In [0]:
import shutil

from cord19q.index import Index

# Copy vectors locally for predictable performance
shutil.copy("./data/cord19-300d.magnitude", "./tmp")

# Build the embeddings index
Index.run("./data/articles.sqlite", "./tmp/cord19-300d.magnitude")

ModuleNotFoundError: ignored